TODO :
 - reorder cells to facilitate testing part
 - do the dataprocess only on the train dataset

In [328]:
%pip install pandas==1.5.1
%pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [327]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [329]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from torchvision.transforms.functional import hflip
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from matplotlib import image as mpimg
import re
from PIL import Image
import pandas as pd
import datetime
from torchmetrics import TweedieDevianceScore

import math
from torch import default_generator, randperm
from torch._utils import _accumulate
from torch.utils.data.dataset import Subset

from sklearn.metrics import fbeta_score, make_scorer, mean_poisson_deviance
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import fbeta_score, make_scorer



In [331]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

PATH='/content/drive/MyDrive/MLproject/'

with open(PATH+"listings.pkl", "rb") as fh:
    data = pickle.load(fh)
with open(PATH+"test_listings_overall.pkl", "rb") as fh:
    data_pred = pickle.load(fh)


images_data_train = pd.read_csv(PATH + "images_vectorized_2.csv")
images_data_pred = pd.read_csv(PATH + "images_vectorized_test.csv")

cuda


In [360]:
# #We load the model
model_saved = torch.load(PATH + 'model.pkl')
model_saved.eval()
model_saved

PredictModel(
  (layer_1): Linear(in_features=65, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=128, bias=True)
  (layer_4): Linear(in_features=128, out_features=64, bias=True)
  (layer_5): Linear(in_features=64, out_features=32, bias=True)
  (layer_out): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
)

In [333]:
images_data_pred

,Property Reference Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,8341.61.6570,-0.849969,-3.240616,-3.917287,2.699604,3.116759,-3.104941,-0.207285,0.774784,3.297490,3.013693,-0.367227,-3.211585,0.591526,-0.318698,-1.845122,3.721253
1,8341.61.6570,-0.133047,-3.502283,-4.219102,3.164258,3.487994,-3.710566,-0.473542,0.992200,3.729333,3.157966,0.318890,-3.499655,0.582798,-0.804382,-2.050940,4.252655
2,8341.61.6570,1.016204,-4.104066,-5.018961,3.695715,4.281897,-4.371731,-1.065704,1.375457,4.286962,3.913172,0.700830,-4.118676,-0.227138,-1.121138,-2.716482,4.860582
3,8341.61.6570,0.114330,-2.968678,-3.761842,2.179164,2.589720,-3.011276,-2.045416,0.024885,3.092517,2.627254,-2.022895,-3.039211,-0.562394,1.104918,-1.223571,3.618255
4,8341.61.6570,-0.715925,-3.227824,-3.849418,2.405258,2.858965,-3.181899,-1.430426,0.314886,3.193606,2.833800,-1.533805,-3.292569,-0.324218,0.720005,-1.580568,3.606343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21166,1926.61.5829,0.685632,-4.020181,-4.698476,2.822384,3.578196,-4.061289,-4.121346,0.188569,4.061197,3.434967,-2.519817,-4.018798,-1.986176,1.315821,-1.878864,4.576988
21167,1926.61.5829,0.574127,-4.806489,-5.569428,4.195199,4.792581,-5.033579,-0.658546,1.354678,4.941759,4.282148,0.315689,-4.570895,0.536657,-0.821080,-2.993514,5.525500
21168,1926.61.5829,-0.749949,-3.481339,-4.220977,2.761591,2.961878,-3.360797,-1.698399,0.626296,3.436264,3.036188,-1.304290,-3.355610,0.020404,0.141869,-1.917535,3.993630
21169,1926.61.5829,0.324329,-6.065782,-6.730525,5.309654,6.473758,-6.336216,-1.905841,2.380155,6.104659,5.530360,1.604911,-5.783335,-0.297243,-2.312171,-4.473103,6.824511


# Dataset

In [350]:
data_pred.columns

Index(['Are Pets Allowed', 'Category Idx', 'Floor', 'Geo City', 'Geo Zip',
       'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace',
       'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Jahr', 'Leerwohnungsziffer',
       'Listing Description', 'Listing Title', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Property Reference Id',
       'Size M2 Normalized', 'Subcategory En Idx', 'Year Built',
       'Year Lastrenovated', 'Prediction', 'Geo_AG', 'Geo_AR', 'Geo_BE',
       'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR', 'Geo_JU',
       'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO', 'Geo_SZ',
       'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG', 'Geo_ZH',
       

In [335]:
features = data[['Are Pets Allowed', 'Category Idx', 'Day of Advertisement Created',
       'Day of Date Available From', 'Floor', 'Geo Canton',
       'Geo City', 'Geo Zip', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Jahr', 'Leerwohnungsziffer',
       'Listing Description', 'Listing Title', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Property Reference Id',
       'Size M2 Normalized', 'Subcategory En Idx', 'Year Built',
       'Year Lastrenovated']]

label = data['Demand']


# Real preprocessing

In [336]:
def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)


index = 'Property Reference Id'

def preprocess(df_input):
  binary_features = ['Are Pets Allowed', 'Has Balcony', 'Has Cabletv', 'Has Elevator',
                  'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
                  'Is New Construction Potential', 'Is Tenant2Tenant',
                  'Is Wheelchairaccessible']
  for column in binary_features:
    df_input.loc[df_input[column] == False, column] = 0
    df_input.loc[df_input[column] == True, column] = 1

  df_input = pd.get_dummies(df_input, prefix=['Geo'], columns=['Geo Canton'])

  df_input['Year Lastrenovated'].fillna(0, inplace = True)

  threshold=datetime.datetime(2022,9,1)
  df_input['Days Between September and Created']=pd.to_datetime(df_input['Day of Advertisement Created']).apply(lambda x:(threshold-x).days)
  df_input['Days Between September and Available']=pd.to_datetime(df_input['Day of Date Available From']).apply(lambda x:(threshold-x).days)                                                           
  df_input.drop(['Day of Advertisement Created','Day of Date Available From'],axis=1,inplace=True)

  df_input['Leerwohnungsziffer'] = pd.to_numeric(df_input['Leerwohnungsziffer'])
  df_input['Jahr'] = pd.to_numeric(df_input['Jahr'])

  input_cols = df_input[['Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator',
       'Has Fireplace', 'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms', 'Number Of Rooms Cleaned',
       'Number of Documents', 'Number of Images', 'Price Extra Normalized',
       'Price Gross Normalized', 'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized', 
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr',
       'Geo_AG', 'Geo_AR', 'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 
       'Geo_GL', 'Geo_GR', 'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 
       'Geo_SH', 'Geo_SO', 'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG', 'Geo_ZH',
       'Days Between September and Created', 'Days Between September and Available' ]]

  # Normalization of feature columns

  input_cols_norm = mean_norm(input_cols)
  input_cols_norm.fillna(input_cols_norm.median(), inplace=True)

  # input_cols_norm['Day of Advertisement Created'] = df_input['Day of Advertisement Created']
  # input_cols_norm['Day of Date Available From'] = df_input['Day of Date Available From']

  first_column = input_cols_norm.pop('Days Between September and Created')
  input_cols_norm.insert(0, 'Days Between September and Created', first_column)
  second_column = input_cols_norm.pop('Days Between September and Available')
  input_cols_norm.insert(1, 'Days Between September and Available', second_column)

  input_cols_norm['Property Reference Id'] = df_input[index]
  
  return df_input, input_cols_norm


data, input_cols_norm = preprocess(data)
data_pred, input_cols_norm_pred = preprocess(data_pred)

output_cols = list(data['Demand'])
input_cols_norm['Demand'] = data['Demand']
input_cols_norm_pred['Demand'] = np.nan


In [337]:
input_cols_norm_pred

,Days Between September and Created,Days Between September and Available,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Property Reference Id,Demand
0,0.091299,0.098188,1.202155,-0.606222,-0.212815,0.767208,0.812694,-0.160562,0.905138,1.128438,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,8469.26.3115,NaN
1,-0.859967,-0.860638,1.202155,-0.606222,-0.212815,0.767208,0.812694,-0.160562,0.905138,1.128438,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,5923.03.2935,NaN
2,-0.871711,-0.860638,-0.831687,-0.133650,-0.212815,0.767208,0.812694,-0.160562,0.905138,-0.886016,...,-0.108799,-0.125057,-0.154785,-0.043628,3.913014,-0.093133,-0.106422,-0.734569,5898.06.5664,NaN
3,1.542526,1.614394,-0.831687,2.701784,-0.212815,0.767208,0.812694,-0.160562,-1.104616,1.128438,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,6321.61.6570,NaN
4,0.010769,0.098188,1.202155,0.338923,-0.212815,0.767208,0.812694,-0.160562,0.905138,1.128438,...,-0.108799,-0.125057,-0.154785,-0.043628,3.913014,-0.093133,-0.106422,-0.734569,3036.61.1151,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.311001,1.360587,1.202155,2.229212,-0.212815,0.767208,0.812694,-0.160562,0.905138,1.128438,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,3743.61.8724,NaN
9996,0.207062,0.201038,-0.831687,-1.078794,-0.212815,0.767208,0.812694,-0.160562,0.905138,-0.886016,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,1.361206,2783.47.3282,NaN
9997,-1.375027,-1.368252,-0.831687,0.338923,-0.212815,0.767208,0.812694,-0.160562,0.905138,-0.886016,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,1.361206,9988.68.0891,NaN
9998,-1.073038,-1.011595,-0.831687,0.811495,-0.212815,0.767208,0.812694,-0.160562,0.905138,-0.886016,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,1.361206,8929.92.2004,NaN


In [338]:
input_cols_norm_pred[ input_cols_norm_pred['Property Reference Id'] == "8341.61.6570" ]


,Days Between September and Created,Days Between September and Available,Are Pets Allowed,Floor,Has Balcony,Has Cabletv,Has Elevator,Has Fireplace,Has Garage,Has Parking,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Property Reference Id,Demand
6862,-1.311274,-1.011595,-0.831687,2.701784,-0.212815,0.767208,0.812694,-0.160562,0.905138,1.128438,...,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,8341.61.6570,NaN


# Merge with image dataset



In [340]:
images_data_train.columns

Index(['Property Reference Id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', 'Demand'],
      dtype='object')

In [342]:
images_data_pred.columns

Index(['Property Reference Id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15'],
      dtype='object')

In [343]:
def merge_dataset(tabular_data, images_data):

  images_id = images_data['Property Reference Id'].unique()
  # print("image_id:", images_id)
  # print("len(images_id):", len(images_id))

  tabular_data = tabular_data[tabular_data['Property Reference Id'].isin(images_id)]

  images_data['Property Reference Id'] = images_data['Property Reference Id'].astype('category')
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')

  images_data.drop('Demand', inplace=True, axis=1)
  data_joined = images_data.merge(tabular_data, on='Property Reference Id', how='left')

  return data_joined


def merge_dataset_test(tabular_data, images_data):

  images_id = images_data['Property Reference Id'].unique()
  # print("image_id:", images_id)
  # print("len(images_id):", len(images_id))

  tabular_data = tabular_data[tabular_data['Property Reference Id'].isin(images_id)]

  images_data['Property Reference Id'] = images_data['Property Reference Id'].astype('category')
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')

  data_joined = images_data.merge(tabular_data, on='Property Reference Id', how='left')

  return data_joined


data_joined = merge_dataset(input_cols_norm, images_data_train)
data_joined_pred = merge_dataset_test(input_cols_norm_pred, images_data_pred)


<ipython-input-343-a9275a66048c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')
<ipython-input-343-a9275a66048c>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabular_data['Property Reference Id'] = tabular_data['Property Reference Id'].astype('category')


In [345]:
data_joined.columns
data_joined_pred.columns

Index(['Property Reference Id', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15',
       'Days Between September and Created',
       'Days Between September and Available', 'Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace',
       'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms',
       'Number Of Rooms Cleaned', 'Number of Documents', 'Number of Images',
       'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_

In [358]:
data_joined_pred

,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SO,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand
0,8341.61.6570,-0.849969,-3.240616,-3.917287,2.699604,3.116759,-3.104941,-0.207285,0.774784,3.297490,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,NaN
1,8341.61.6570,-0.133047,-3.502283,-4.219102,3.164258,3.487994,-3.710566,-0.473542,0.992200,3.729333,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,NaN
2,8341.61.6570,1.016204,-4.104066,-5.018961,3.695715,4.281897,-4.371731,-1.065704,1.375457,4.286962,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,NaN
3,8341.61.6570,0.114330,-2.968678,-3.761842,2.179164,2.589720,-3.011276,-2.045416,0.024885,3.092517,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,NaN
4,8341.61.6570,-0.715925,-3.227824,-3.849418,2.405258,2.858965,-3.181899,-1.430426,0.314886,3.193606,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,-0.255532,-0.093133,-0.106422,-0.734569,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29814,1926.61.5829,0.685632,-4.020181,-4.698476,2.822384,3.578196,-4.061289,-4.121346,0.188569,4.061197,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,3.913014,-0.093133,-0.106422,-0.734569,NaN
29815,1926.61.5829,0.574127,-4.806489,-5.569428,4.195199,4.792581,-5.033579,-0.658546,1.354678,4.941759,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,3.913014,-0.093133,-0.106422,-0.734569,NaN
29816,1926.61.5829,-0.749949,-3.481339,-4.220977,2.761591,2.961878,-3.360797,-1.698399,0.626296,3.436264,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,3.913014,-0.093133,-0.106422,-0.734569,NaN
29817,1926.61.5829,0.324329,-6.065782,-6.730525,5.309654,6.473758,-6.336216,-1.905841,2.380155,6.104659,...,-0.189305,-0.108799,-0.125057,-0.154785,-0.043628,3.913014,-0.093133,-0.106422,-0.734569,NaN


# Data to tensors

In [349]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.autograd import Variable
import torchvision.models as models

In [372]:
# #TRAIN PART ------------------------------------------------------------------------------------------------------------------------------------

# label = data_joined_train['Demand']
# output_cols = list(label)

# input_cols = data_joined_train[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
#        '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
#        'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
#        'Has Parking', 'Is New Construction', 'Is New Construction Potential',
#        'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
#        'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
#        'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
#        'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
#        'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
#        'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
#        'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
#        'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
#        'Geo_ZH']]


#TEST PART -----------------------------------------------------------------------------------------------------------------------------------

label_test = data_joined_pred['Demand']
output_cols_test = list(label_test)

input_cols_pred = data_joined_pred[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
       'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
       'Has Parking', 'Is New Construction', 'Is New Construction Potential',
       'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       'Geo_ZH']]

pred_id = data_joined_pred['Property Reference Id']

In [364]:
input_cols_pred.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Are Pets Allowed', 'Floor', 'Has Balcony',
       'Has Cabletv', 'Has Elevator', 'Has Fireplace', 'Has Garage',
       'Has Parking', 'Is New Construction', 'Is New Construction Potential',
       'Is Tenant2Tenant', 'Is Wheelchairaccessible', 'Livingspace',
       'Number Of Rooms', 'Number Of Rooms Cleaned', 'Number of Documents',
       'Number of Images', 'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       'Geo_ZH'],
      dtype='object')

In [365]:
inputs_array_pred = np.array(input_cols_pred.to_numpy())
# targets_array1_pred = label_test.to_numpy()

# targets_array_pred=[]
# We add brakets for tensor structure... (can be done differently)

# for elt in targets_array1_pred:
#     targets_array_pred.append([elt])
# targets_array_test = np.array(targets_array_pred)

# print(type(inputs_array_pred))
# print(type(targets_array_pred))
# print(inputs_array_pred)
# print(targets_array_pred)

inputs_pred = torch.from_numpy(inputs_array_pred).to(torch.float32)




# targets_pred = torch.from_numpy(targets_array_test).to(torch.float32)
# dataset_pred = TensorDataset(inputs_pred, targets_pred)
# print("test dataset:", dataset_pred)




In [375]:
predictions = model_saved(inputs_pred).cpu().detach().numpy().reshape(len(model_saved(inputs_pred)))
print(predictions)

[ 3.1284535  3.1753943  3.731882  ...  9.83727   10.6533165 12.156069 ]


In [377]:
pred_df = pd.DataFrame(np.array([pred_id, predictions]).T, columns = ["Property Reference Id", "Prediction"])
pred_df

,Property Reference Id,Prediction
0,8341.61.6570,3.128453
1,8341.61.6570,3.175394
2,8341.61.6570,3.731882
3,8341.61.6570,3.084382
4,8341.61.6570,3.32178
...,...,...
29814,1926.61.5829,11.792503
29815,1926.61.5829,9.485902
29816,1926.61.5829,9.83727
29817,1926.61.5829,10.653316


# Model


In [315]:
#Creation of linear regression model

in_cols = input_cols.columns
out_cols = ['Demand']

input_size = len(in_cols)
output_size = len(out_cols)

class PredictModel(nn.Module):


    def __init__(self):
        # super().__init__()
        # self.linear = nn.Linear(input_size, output_size)
        super(PredictModel, self).__init__()
        self.layer_1 = nn.Linear(input_size, 32)
        self.layer_2 = nn.Linear(32, 64)
        self.layer_3 = nn.Linear(64, 128)
        # self.layer_4 = nn.Linear(64, 128)
        # self.layer_5 = nn.Linear(128, 64)
        self.layer_4 = nn.Linear(128, 64)
        self.layer_5 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, output_size)
        
        self.relu = nn.ReLU()
    
    def forward(self, xb):
        # out = self.linear(xb)
        x = self.relu(self.layer_1(xb))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.relu(self.layer_4(x))
        x = self.relu(self.layer_5(x))
        # x = self.relu(self.layer_6(x))
        # x = self.relu(self.layer_7(x))
        x = self.layer_out(x)
        return x


    def training_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        out = self(inputs)
        loss = F.l1_loss(out, targets)
        return {'val_loss': loss.detach()}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        if (epoch+1) % 20 == 0 or epoch == num_epochs - 1:
            print("Epoch[{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [316]:
#Defining our model 
model = PredictModel()
list(model.parameters())

#Training loop
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history=[]
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

result = evaluate(model, val_loader)
print(result)




{'val_loss': 3.8297526836395264}


In [317]:
epochs = 300
lr = 0.001
history = fit(epochs, lr, model, train_loader, val_loader)

Epoch[20], val_loss: 3.3626
Epoch[40], val_loss: 3.3768
Epoch[60], val_loss: 3.3601
Epoch[80], val_loss: 3.3396
Epoch[100], val_loss: 3.2916
Epoch[120], val_loss: 3.1876
Epoch[140], val_loss: 3.1560
Epoch[160], val_loss: 2.9818
Epoch[180], val_loss: 3.0018
Epoch[200], val_loss: 2.9023
Epoch[220], val_loss: 2.9416
Epoch[240], val_loss: 3.0160
Epoch[260], val_loss: 2.9281
Epoch[280], val_loss: 2.9949
Epoch[300], val_loss: 2.8482


In [318]:
# We try to test our model on a random value of the test set

def predict_single(input, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)         
    prediction = predictions[0].detach()
    return prediction

input, target = val_ds[50]
prediction = predict_single(input, model)

print("Input:", input)
print("Target:", target)
print("Prediction:", prediction)
print()


# Return the entire predictions (Tensor Y_label)

def predict_all(val_ds, model):
    predictions = []
    targets = []
    for i in range(len(val_ds)):
       input, target = val_ds[i]
       prediction = predict_single(input, model)
       predictions.append(prediction.numpy()[0])
       targets.append(target)
    return predictions, targets

predictions, targets = predict_all(val_ds, model)

#On arrondi à l'entier le plus proche pour effectuer notre prédiction et on remplace les valeurs négatives

predictions = [max(10**(-0.05),k) for k in predictions]



Input: tensor([ 2.0991,  6.5296,  5.8832, -7.4918,  4.1345, -1.8794,  2.2303,  2.1299,
        -0.6852,  7.0435, -9.7505,  1.4381, -3.4204, -7.5194,  8.5891, -0.7026,
        -0.8322, -1.0675, -0.2002,  0.7588,  0.7989, -0.1561,  0.8999, -0.9166,
        -0.0794, -0.1257, -0.1090, -0.0535, -1.6130, -1.2049, -1.2049, -1.4841,
         1.5834, -0.7723, -0.7328,  0.0682, -0.6882, -1.6134, -0.9030, -0.7763,
         1.6861, -0.3558, -0.0490, -0.2556, -0.2370, -0.1906, -0.1600, -0.2108,
        -0.0436, -0.0723, -0.0122, -0.2217, -0.1367, -0.0464, -0.2688, -0.0997,
        -0.1968, -0.0971, -0.1323, -0.1477, -0.0511,  3.8516, -0.1050, -0.1047,
        -0.7235])
Target: tensor([3.])
Prediction: tensor([3.6799])



In [319]:
print(predictions)

[0.8912509381337456, 0.8912509381337456, 11.951239, 12.862627, 15.712616, 16.296003, 12.629386, 13.323881, 13.058647, 13.761873, 11.821325, 12.539836, 12.963602, 13.8610525, 14.155253, 14.749472, 11.8868265, 12.580972, 12.421556, 13.256086, 14.441715, 15.023104, 11.502541, 12.197082, 12.113951, 12.7645, 12.564449, 13.25085, 12.341272, 13.043658, 4.789196, 4.846039, 4.5992837, 4.524922, 4.4307737, 4.377219, 5.103767, 5.0122423, 4.4252124, 4.3223305, 4.789892, 4.7584004, 4.2447686, 4.3309455, 5.213869, 5.1195693, 4.335188, 4.4261765, 4.0015216, 4.0752773, 3.679859, 3.750419, 4.5811124, 4.653085, 4.3875127, 4.4721885, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.8912509381337456, 0.891250938133745

In [320]:
#Computation of the accurcy of our model on test set
EPSILON = 2

def accuracy(val_ds, model, EPSILON):
    corrects = 0
    for i in range(len(val_ds)):
        input, target = val_ds[i]
        prediction = predict_single(input, model)
        prediction = np.round(prediction)
        if prediction <= target + EPSILON and prediction >= target - EPSILON:
            corrects+=1
    accuracy = corrects/len(val_ds)
    print("Accuracy with EPSILON={}:".format(EPSILON), accuracy)
    return accuracy

accuracy(val_ds, model, EPSILON)



#Using Mean Poisson deviance
from sklearn.metrics import mean_poisson_deviance

print("Mean Poisson deviance:", mean_poisson_deviance(targets, predictions))




Accuracy with EPSILON=2: 0.6991452991452991
Mean Poisson deviance: 4.439334759797035


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:746: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)


# We groupby images by ID

In [321]:
def groupbyID(data_joined_test):

  # We add the prediction on the test set to gather our results order by id
  data_joined_test['Prediction'] = predictions

  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)

  temp_dataset = data_joined_test.groupby(['Property Reference Id'])
  results = temp_dataset.mean()

  data_joined_test.loc[data_joined_test.Prediction<=0,'Prediction']=10**(-0.05)

  return results

results = groupbyID(data_joined_test)

print()
print()
print("Final mean Poisson deviance(per ID):", mean_poisson_deviance(results["Demand"], results["Prediction"]))







Final mean Poisson deviance(per ID): 5.110922386591531


<ipython-input-321-d04d04e1b3f5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test['Prediction'] = predictions
<ipython-input-321-d04d04e1b3f5>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)


In [324]:
results.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'Days Between September and Created',
       'Days Between September and Available', 'Are Pets Allowed', 'Floor',
       'Has Balcony', 'Has Cabletv', 'Has Elevator', 'Has Fireplace',
       'Has Garage', 'Has Parking', 'Is New Construction',
       'Is New Construction Potential', 'Is Tenant2Tenant',
       'Is Wheelchairaccessible', 'Livingspace', 'Number Of Rooms',
       'Number Of Rooms Cleaned', 'Number of Documents', 'Number of Images',
       'Price Extra Normalized', 'Price Gross Normalized',
       'Price M2 Normalized', 'Price Net Normalized', 'Size M2 Normalized',
       'Year Lastrenovated', 'Leerwohnungsziffer', 'Jahr', 'Geo_AG', 'Geo_AR',
       'Geo_BE', 'Geo_BL', 'Geo_BS', 'Geo_FR', 'Geo_GE', 'Geo_GL', 'Geo_GR',
       'Geo_JU', 'Geo_LU', 'Geo_NE', 'Geo_NW', 'Geo_SG', 'Geo_SH', 'Geo_SO',
       'Geo_SZ', 'Geo_TG', 'Geo_TI', 'Geo_UR', 'Geo_VD', 'Geo_VS', 'Geo_ZG',
       

# Save and load the model

In [ ]:
model_saved()

## 1st version of groupby

In [ ]:
# We add the prediction on the test set to gather our results order by id
data_joined_test['Prediction'] = predictions
data_joined_test.sample(5)


<ipython-input-75-b8612112f83e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test['Prediction'] = predictions


,Property Reference Id,0,1,2,3,4,5,6,7,8,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Prediction
6787,7684.61.7705,-0.502878,5.947612,5.516549,-6.756667,3.567677,-2.863783,1.876810,4.323555,1.001609,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,1.0,0.891251
446,0668.61.0065,-0.463543,5.042772,5.045663,-6.140898,2.529379,-2.926522,1.945866,4.458686,1.526753,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,14.0,1.026594
7252,8341.50.7705,-0.955973,4.044521,4.508246,-4.677773,1.006604,0.447340,0.144032,2.436692,1.052145,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,14.0,1.254935
2125,2490.22.0964,-1.118535,4.259557,4.679176,-4.985484,1.229805,0.267612,-0.101243,2.276172,1.229883,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.0,2.042334
7850,9179.61.4593,-1.852610,5.425081,5.757202,-6.920499,2.561271,-1.741298,-1.045670,-1.061599,2.489897,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,1.0,0.891251


In [ ]:
data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)
data_joined_test.dtypes


<ipython-input-42-342d6ca3f86a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_joined_test["Property Reference Id"] = data_joined_test["Property Reference Id"].astype(object)


Property Reference Id     object
0                        float64
1                        float64
2                        float64
3                        float64
                          ...   
Geo_VS                   float64
Geo_ZG                   float64
Geo_ZH                   float64
Demand                   float64
Prediction               float64
Length: 70, dtype: object

In [ ]:
temp_dataset = data_joined_test.groupby(['Property Reference Id'])
tab = temp_dataset.mean()
display(tab.sample(5))

<ipython-input-43-36475d7a97ee>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tab = temp_dataset.mean()


,0,1,2,3,4,5,6,7,8,9,...,Geo_SZ,Geo_TG,Geo_TI,Geo_UR,Geo_VD,Geo_VS,Geo_ZG,Geo_ZH,Demand,Prediction
Property Reference Id,,,,,,,,,,,,,,,,,,,,,
7276.22.0065,-0.151769,4.691804,4.763517,-5.378359,2.089910,-1.122222,0.730841,1.588573,0.597662,5.687003,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,0.000000,0.891251
7977.22.5664,0.213037,4.904241,4.894507,-5.553480,2.216729,-0.663933,0.927539,1.938606,0.325356,5.876572,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,2.000000,1.670750
5888.22.9935,0.472074,5.079700,4.953190,-5.986607,2.689590,-1.171054,1.182277,2.387276,0.408414,5.914227,...,-0.097148,-0.132288,-0.147698,-0.051055,3.851567,-0.104974,-0.10473,-0.723488,2.000000,0.891251
6791.61.4266,0.016830,4.677358,4.784645,-5.300134,1.915972,-0.340583,0.822994,2.344957,0.440216,5.762862,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,4.333333,1.232676
9564.26.4593,-0.107945,5.242588,5.112111,-6.010555,2.570060,-0.540652,0.367418,1.305161,0.755894,6.135576,...,-0.097148,-0.132288,-0.147698,-0.051055,-0.259622,-0.104974,-0.10473,-0.723488,8.000000,2.948352


In [214]:
data_joined_test.loc[data_joined_test.Prediction<=0,'Prediction']=10**(-0.05)
print("Final mean Poisson deviance(per ID):", mean_poisson_deviance(data_joined_test["Demand"], data_joined_test["Prediction"]))


Final mean Poisson deviance(per ID): 5.256561406907189
